# Майнор "Интеллектуальный анализ данных", Введение в Анализ данных (ВШЭ). Семинар 5: библиотека sklearn и метод ближайшего соседа

In [4]:
import numpy as np

In [10]:
import sklearn

### scikit-learn: machine learning in Python 
В библиотеке __sklearn__ реализованы все основные методы машинного обучения. Для них предусмотрены удобные унифированные интерфейсы (и их копируют другие библиотеки) и достаточно эффективные алгоритмы (хотя, как правило, sklearn используют для выбора алгоритма и модели, а затем их уже пишут на низкоуровневых языках с целью повышения скорости работы). Кроме того, sklearn предоставляет некоторые дополнительные средства, в частности, для предобработки данных и для выбора модели.

http://scikit-learn.org/

Обратите внимание на многообразие документации: быстрый старт, полная документация (Usrr Guide), tutuorials (для решения стандартных задач), API (методы, классы, параметры) и красивый flowchart.

## Любите документацию sklearn, читайте ее, наслаждайтесь ей! Подсматривайте в ней интерфейсы методов и делайте новые открытия каждый день :D

### Примеры датасетов

In [20]:
from sklearn.datasets import load_boston
boston = load_boston()

In [16]:
" ".join(dir(boston))

'DESCR __class__ __cmp__ __contains__ __delattr__ __delitem__ __dict__ __doc__ __eq__ __format__ __ge__ __getattribute__ __getitem__ __gt__ __hash__ __init__ __iter__ __le__ __len__ __lt__ __module__ __ne__ __new__ __reduce__ __reduce_ex__ __repr__ __setattr__ __setitem__ __sizeof__ __str__ __subclasshook__ __weakref__ clear copy data feature_names fromkeys get has_key items iteritems iterkeys itervalues keys pop popitem setdefault target update values viewitems viewkeys viewvalues'

In [18]:
" | ".join(boston.keys())

'data | feature_names | DESCR | target'

In [22]:
print boston.data.shape

(506L, 13L)


В sklearn всегда (как и в теории) строки --- это объекты, столбцы --- это признаки

In [24]:
print boston.data.min(axis=0)
print boston.data.max(axis=0)
print boston.data.mean(axis=0)
# В pandas это можно было сделать одной строчкой кода, и еще мы бы знали названия признаков...

[  6.32000000e-03   0.00000000e+00   4.60000000e-01   0.00000000e+00
   3.85000000e-01   3.56100000e+00   2.90000000e+00   1.12960000e+00
   1.00000000e+00   1.87000000e+02   1.26000000e+01   3.20000000e-01
   1.73000000e+00]
[  88.9762  100.       27.74      1.        0.871     8.78    100.
   12.1265   24.      711.       22.      396.9      37.97  ]
[  3.59376071e+00   1.13636364e+01   1.11367787e+01   6.91699605e-02
   5.54695059e-01   6.28463439e+00   6.85749012e+01   3.79504269e+00
   9.54940711e+00   4.08237154e+02   1.84555336e+01   3.56674032e+02
   1.26530632e+01]


In [27]:
# но названия признаков можно посмотреть в описании
print boston.DESCR

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

Другие датасеты в sklearn.datasets

In [32]:
import sklearn.datasets
" | ".join(dir(sklearn.datasets))

'__all__ | __builtins__ | __doc__ | __file__ | __name__ | __package__ | __path__ | _svmlight_format | base | california_housing | clear_data_home | covtype | dump_svmlight_file | fetch_20newsgroups | fetch_20newsgroups_vectorized | fetch_california_housing | fetch_covtype | fetch_lfw_pairs | fetch_lfw_people | fetch_mldata | fetch_olivetti_faces | fetch_species_distributions | get_data_home | lfw | load_boston | load_diabetes | load_digits | load_files | load_iris | load_lfw_pairs | load_lfw_people | load_linnerud | load_mlcomp | load_sample_image | load_sample_images | load_svmlight_file | load_svmlight_files | make_biclusters | make_blobs | make_checkerboard | make_circles | make_classification | make_friedman1 | make_friedman2 | make_friedman3 | make_gaussian_quantiles | make_hastie_10_2 | make_low_rank_matrix | make_moons | make_multilabel_classification | make_regression | make_s_curve | make_sparse_coded_signal | make_sparse_spd_matrix | make_sparse_uncorrelated | make_spd_matrix

### Обучение с учителем

Для каждого метода в sklearn реализован отдельный класс, например, sklearn.neighbors.KNeighborsClassifier. 
При вызове его конструктора необходимо указать параметры метода или алгоритма обучения (но для них, как правило, есть значения по умолчанию). 

Для того, чтобы обучить модель по данным, нужно воспользоваться методом __fit(X, y)__, где X --- матрица объекты-признаки, y - значение целевой переменной (дискретная для классификации и непрерывная для регрессии). Обязательное условие: X.shape[0] == y.shape[0] (в i-й строке y находится правильный ответ для i-го объекта в X). В результате в модели настроятся параметры, необходимые для предсказаний).

Чтобы сделать предсказание на новых данных, нужно воспользоваться методом __predict(X)__, X --- матрица объекты-признаки. Метод вернет вектор y.

Часто класс также предоставляет метод __predict_proba(X)__, который возвращает ответы и степень уверенности классификатора или регрессора в них (вероятность того, что объект принадлежит какому-то классу). Более конкретно, для классификации метод возвращает матрицу размера число объектов x число классов, состоящую из таких вероятностей.

Кроме того, у каждого класса есть методы __get_params()__, который вернет словарь параметров (тех, которые указываются в конструкторе) и __set_params(**params)__, который может менять значения параметров.

### Обучение без учителя (кластеризация)

Для кластеризации интерфейс похожий, но из-за отсутствия целевой переменной в fit нужно подавать только матрицу X. 

Также есть метод fit_predict(X), который находит кластеры и возвращает вектор принадлежности объектов кластерам.

In [ ]:
# Сохранение построенной модели
import pickle
with open("file.txt", "w") as f_out:
    pickle.dump(clf, f_out)

При работе с реальными данными можно попасть в ситацию, когда вы запускаете обучение модели и не видите прогресс в обучении (cell долго работает, и вы не знаете, когда он закончит работать). Поэтому, при возникновении подобной ситуации, рекомендуется запустить обучение несколько раз на совсем маленьких выборках, чтобы понять, как время обучения зависит от размера входных данных (например, построить график), и уже оценив его для всей выборки, решать, сможете ли вы так долго ждать обучения модели. 

Такая же ситуация может возникнуть и в методом predict.

Далее мы будем изучать, как все эти методы работают. И начнем с наиболее простой группы методов, на примере которой еще раз вспомним основные аспекты решения задач машинного обучения.

## Метрические классификаторы

В машинном обучении распространено понятие __эвристики__ --- некоторой идеи, которая кажется правдоподобной (хотя не имеет четкого обоснования) и на которой может основываться метод или улучшение какого-то метода.

#### Напоминание: обучение модели
Задача машинного обучения выглядит так: дана обучающая выборка в виде матрицы объекты-признаки и задан какой-то вопрос относительно объектов; нужно научиться отвечать на это вопрос для новых объектов. Для этого придумывают модель, имеющую некоторые параметры, которые настраивают с помощью оптимизации какого-то критерия. Кроме того, у модели могут быть гиперпараметры, не настраиваемые в процессе обучения, которые нужно задавать самостоятельно.

#### Напоминание: шумовой объект
В машинном обучении приходится работать с совершенно неидеальными данными, в которых увидеть закономерность можно только по достаточно большой выборке. В этой выборке могут встречаться объкты, которые противоречат основной тенденции остальных объектов (например, деревня по многим показателям похожа на город). Такие объекты называют _шумовыми_, или _выбросами_.

#### Напоминание: переобучение
Переобучением называется явление, когда алгоритм выдает маленькую ошибку на обучающей выборке, но большую ошибку на новых данных. В этом случае алгоритм настраивается на шум в данных и перестает улавливать закономерности.

#### Напоминание: задача классификации

Дана обучающая выборка (матрица объекты-признаки), выделен один признак (целевой), у которого множество значений конечно (например, цифры, категории, бинарный ответ да/нет). Этот признак мы и хотим предсказывать для новых объектов. Классификация может быть бинарной и многоклассовой.

__Простая эвристика__: давайте выдавать такой же ответ, какой является правильным для объекта обучающей выборки, наиболее похожего на данный (это будет наша _модель ответа_). 
Такой классификатор называется __Метод ближайшего соседа__.

В его основе лежит _предположение_ о том, что объекты одного класса лежат близко друг к другу в пространстве признаков (_гипотеза компактности_).

Стоит обратить внимание, что это чуть ли не единственный метод машинного обучения, в котором процедура _обучения_ состоит в _запоминании_ выборки (не нужно ничего оптимизировать).

Например, легко понять, что для двумерного случая классификация всей плоскости задается разбиением ее на многоугольники, ограниченные серединными перпендикулярами к отрезкам, попарно соединяющим объекты обучающей выборки.
Это показано на левой картинке (здесь выборка состоит из 8 объектов, целевой признак --- цвет точки: синий или зеленый, каждый объект представлен двумя признаками-координатами на вещественной плоскости).

![](https://shapeofdata.files.wordpress.com/2013/03/knn1.png)

Здесь кроется и вся опасность МБС --- его склонность к переобучению: для любого шумового объекта (например, правой нижней синей точки) будет создаваться отдельная область, то есть алгоритм крайне чувствителен к шуму.

Давайте немного усложним классификатор: вместо одного соседа будем рассматривать k наиболее похожих на данный объект, и выбирать тот класс, который наиболее часто встречается среди k соседей (новая _модель_). Получится классификатор, более устойчивый к переобучению. Этот метод называется __метод k ближайших соседей, или KNN__. Например, на правой картинке показана разделяющая поверхность при k=3, и там уже синий объект признается шумовым.

Очевидно, число соседей k является _гиперпараметром_ модели, который нужно задавать перед обучением. Для этого, как правило, используют кросс-валидацию (о ней чуть-чуть упоминали на лекциях, и мы позже разберем на семинарах).

### Обсуждение kNN

Для ряда задач (например, для распознавания цифр или классификации текстов, представленных в виде <<мешка слов>>), kNN дает хороший результат. Ключевой вопрос состоит в __выборе метрики для сравнения объектов__. Для вещественных объектов $x = (x_1, \dots, x_d)$ и $y=(y_1, \dots, y_d)$ используют:
1. расстояние Минковского с показателем $p$: $\rho(x, y) = \biggl(\sum_{i=1}^d (x_i - y_i)^p\biggr)^{\frac 1 p}$; при $p=2$ это привычное _евклидово расстояние_.
1. косинусную метрику:  $\rho(x, y) = \frac {<x, y>} {\sqrt{<x, x><y, y>}}$, $<. ,.>$ --- скаалярное произведение: $<x, y> = \sum_{i=1}^d x_i y_i$. Это расстояние равно косинусу угла между векторами $x$ и $y$ в евклидовом пространстве; хорошо работает для текстов.

Для бинарных векторов (каждый компонент есть 0 или 1):
1. расстояние Хэмминга --- число несовпадающих разрядов. 
1. расстояние Жаккарда --- число единиц в поэлементной конъюнкции, деленное на число элемент в поэлементной дизъюнкции.

Последнее есть интерпретация расстояния Жаккарда для множеств A и B:
1. $\rho(A, B) = \frac {|A \cap B|}{|A \cup B|}$

(Подробнее о метриках здесь: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html)

Если в задаче известны признаки разной природы, можно составлять комбинированную метрику. 

Более того, можно вводить веса признаков $w = (w_1, \dots, w_d)$ и модифицировать метрики, например:
$\rho(x, y) = \sqrt{\sum_{i=1}^d w_i(x_i - y_i)^2}$.

Последний прием важен на _ненормированных_ выборках (если в сумме одно слагаемое имеет порядок 1000, а другое 0.1, то второе не будет иметь влияния на величину расстояния). Тогда в качестве весов можно использовать обратные к средним или к максимальным значениям признаков.

Веса можно применять и к объектам, тогда объекты, находящиеся на расстоянии, более близком к новой точке, будут иметь большее влияние.

### kNN в sklearn

Интерфейс описан в [документации](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [38]:
# импортируем классификатор
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

In [57]:
# загружаем данные --- изображения цифр
from sklearn.datasets import load_digits
data = load_digits()

In [58]:
X = data.images
y = data.target

In [59]:
X.shape

(1797L, 8L, 8L)

In [60]:
X = X.reshape(X.shape[0], -1) # вытягиваем квадратное изображение в вектор, чтобы получить матрицу объекты-признаки

Чтобы избежать переобучения (а также чтобы подобрать k), нужно разделить выборку на train и test. Для этого объекты нужно случайно перемешать и разделить в какой-то пропорции:

In [52]:
from sklearn.utils import shuffle

In [53]:
X, y = shuffle(X, y)

In [54]:
X.shape, y.shape # проверяем, что все хорошо перемешалось

((1797L, 64L), (1797L,))

In [55]:
y[:10] # теперь в случайном порядке

array([1, 4, 8, 6, 5, 2, 5, 6, 5, 6])

In [74]:
X_train, y_train = X[:700, :], y[:700]
X_val, y_val = X[700:1300, :], y[700:1300] #validation
X_test, y_test = X[1300:, :], y[1300:]

In [79]:
# Обучаем классификатор и делаем предсказания
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [80]:
# Вычисляем простейшую метрику качества алгоритма --- долю правильных ответов
print "Accuracy is", np.mean(y_test==y_predicted)

Accuracy is 0.955734406439


Учитывая, что у нас 10 классов, и вероятность случайно вытащить два одинаковых маленькая, точность 0.956 --- очень хороший результат!

In [84]:
# Подбор k на валидационной выборке:
for k in range(1, 20):
    y_predicted = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train).predict(X_val)
    print "k =", k, "; accuracy =", np.mean(y_predicted==y_val)

k = 1 ; accuracy = 0.958333333333
k = 2 ; accuracy = 0.956666666667
k = 3 ; accuracy = 0.956666666667
k = 4 ; accuracy = 0.956666666667
k = 5 ; accuracy = 0.958333333333
k = 6 ; accuracy = 0.958333333333
k = 7 ; accuracy = 0.963333333333
k = 8 ; accuracy = 0.965
k = 9 ; accuracy = 0.965
k = 10 ; accuracy = 0.966666666667
k = 11 ; accuracy = 0.958333333333
k = 12 ; accuracy = 0.961666666667
k = 13 ; accuracy = 0.96
k = 14 ; accuracy = 0.956666666667
k = 15 ; accuracy = 0.951666666667
k = 16 ; accuracy = 0.946666666667
k = 17 ; accuracy = 0.94
k = 18 ; accuracy = 0.94
k = 19 ; accuracy = 0.94


Наилучший результат при k=10

Сравним точность (accuracy) на трех датасетах:

In [86]:
clf = KNeighborsClassifier(n_neighbors=10)
clf.fit(X_train, y_train)
for X_data, y_data in zip([X_train, X_val, X_test], [y_train, y_val, y_test]):
    y_predicted = clf.predict(X_data)
    print "Accuracy:", np.mean(y_predicted==y_data)

Accuracy: 0.987142857143
Accuracy: 0.966666666667
Accuracy: 0.941649899396


Качество на обучающей выборке самое лучшее, но оно обманчиво, ведь алгоритм уже знает эти объекты (переобучение).
На валидационной выборке мы тоже использовали ответы --- уже для подбора гиперпараметра k, так что этот показатель тоже не совсем честный. 
И действительно, качество на тестовой выборке (ответы для которой мы нигде не подсматривали) оказалось хуже, чем на валидационной выборке.

_Вывод_: оценивать качество алгоритма нужно на отложенной выборке, которая не используется нигде в обучении.